# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd # Импортируем библиотеку pandas для работы с датасетом
data = pd.read_csv('/datasets/data.csv')
data.head(10) # посмотрим на данные

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Из первых десяти строк уже видны некоторые проблемы в данных:**
- Количество дней указано не в целочисленном типе и встречаются отрицательные значения
- в столбце образование разные регистры написания
- в столбце с типом гендера английский язык, хотя все остальные данные на русском

In [ ]:
data.info() # запрашиваем информацию о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Из информации** о данных видим что всего у нас 21525 строки, 12 столбцов. Также видно, что в столбцах 'days_employed', 'total_income' имеются пропуски. 

**Описание данных предоставленное от разработчиков:**
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

In [ ]:
# взглянем на названия столбцов
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Проблем в названиях столбцов не обнаружено

**Вывод**

В каждой строке таблицы — данные о заемщиках. Колонка 'purpose' описывает цель получения кредита. Остальные данные рассказывают о заемщике: количество детей, стаж, возраст, уровень образования, семейное положение, пол, тип занятости, имел ли клиент задолженности и ежемесячный доход.

Предварительно можно утверждать, что, данных достаточно для ответа на поставленные вопросы. Но встречаются пропуски в данных и другие проблемы описсанные ранее.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
# подсчитаем количество пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<div class="alert alert-info"> <b>Комментарий студента:</b> Посчитаем долю пропусков:</div>


In [ ]:
data.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

In [ ]:
# Отфильтруем таблицу и найдем строки в которых пропуски в количестве дней и в месячном доходе
data_filtered_isna = data[data['days_employed'].isna()]
data_filtered_isna = data_filtered_isna[data_filtered_isna['total_income'].isna()]
data_filtered_isna

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


**Таким образом**, строки в которых были пропуски в столбце с количеством дней стажа имеют пропуски и в столбце с месячным доходом. Вероятно это произошло при выгрузке датасета, либо эти данные могут не разглашаться.

Причина появления этих пропусков нигде не указана, а составляют строки с пропусками 10% от объема данных. Считаю что полное удаление строк может исказить результаты, поэтому заменим пропуски на характерные значения. 

In [ ]:
# Применим к столбцу с днями функцию abs через метод apply
data['days_employed'] = data['days_employed'].apply(abs)
# Посмотрим на обновленные данные
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Даже из пяти строчек можно предположить что в данных имеются артефакты. Например, в четвертой строке значения дней стажа 340266, а это 932 года.
Посмотрим на данные в датасэте в порядке убывания по столбцу days_employed

In [ ]:
# Сортируем данные по столбцу days_employed
data.sort_values(by='days_employed', ascending=False).head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
4697,0,401635.032697,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
13420,0,401619.633298,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
17823,0,401614.475622,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи
10991,0,401591.828457,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования
8369,0,401590.452231,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование


Поскольку таких данных много, попробуем посчитать их примерное количество с помощью фильтра

In [ ]:
# Создадим фильтр и посчитаем сколько записей в данных имеют странные значения
data[data['days_employed'] > 20000]['days_employed'].count()

3445

Объем подозрительных данных составляет 16% от объема всех данных. 
    
Поскольку пропусков слишком много и такие пропуски как стаж и доход зависят от других факторов - заменять их на одно и то же значение нельзя. Будем искать характерные значения как медианы относительно  типа занятости чтобы скомпенсировать аномальные выбросы.


Посмотрим какие уникальные значения и в каком количестве присутствуют в столбце с типом занятости.

In [ ]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Проблем с регистром и дубликатами в этом столбце нет. Создадим цикл который пройдет по типам занятости, просчитает для каждого типа медианное значение стажа и медианное значение дохода, а потом заполнит пропуски для определенного типа занятости найденными значениями.

p.s. сделал сначала для одного типа вручную, а потом оформил для этого цикл (было сложно, но это кайф, большое спасибо, что объяснил над чем здесь и как поработать)


In [ ]:
# создаем список с типами занятости
income_type_list = ['сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'безработный', 'предприниматель', 'студент', 'в декрете']
# в цикле проходим по всем типам занятости, находим для них медианные значения стажа и дохода, и далее заменяем пропуски для этого типа занятости
for income_type in income_type_list:
    median_income = data[data['income_type'] == income_type]['total_income'].median()
    median_days = data[data['income_type'] == income_type]['days_employed'].median()
    data.loc[data['income_type'] == income_type, 'total_income'] = data.loc[data['income_type'] == income_type, 'total_income'].fillna(median_income)
    data.loc[data['income_type'] == income_type, 'days_employed'] = data.loc[data['income_type'] == income_type, 'days_employed'].fillna(median_days)

data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

В данных были обнаружены пропуски в двух столбцах: количество дней стажа, месячный доход. Последовательно отфильтровав таблицу по этим столбцам я убедился что пропуски по столбцам находятся на одних и тех же строчках. Поскольку количество пропусков составляет практически 10% от объема датасета, то удаление этих строк исказит результаты исследования. Было решено заменить пропуски на характерные значения. Далее я заметил подозрительно большие значения, пранализировал их и решил, что характерные значения для пропусков буду находить через медианы, поскольку на них не повлияют слишком большие и редкие значения. Но брать медианы по всему столбцу нельзя, так как стаж и доход зависят от других факторов. Поэтому было решено искать медианы и менять пропуски относительно типа занятости.

### Замена типа данных

In [ ]:
# вызовем еще раз информацию о датафрейме, чтобы изучить типы данных столбцов
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Из полученной информации видим следующие проблемы:
- в столбце days_employed значения принадлежат типу float
- в столбце total_income значения принадлежат типу float

Преобразуем значения в этих столбцах в тип int

In [ ]:
# преобразование тип float в int в столбцах days_employed и total_income
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [ ]:
# Проверим преобразование типов
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Просмотрев типы данных у столбцов я заметил, что у столбца days_employed и total_income значения принадлежат к типу float. Для удобства анализа и восприятия данных я заменил значения в этих столбцах на тип int при помощи функции astype().

### Обработка дубликатов

Начнем обработку дубликатов с приведения всех столбцов к нижнему регистру


In [ ]:
# Выведем данные для удобства
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [ ]:
# Посчитаем упоминания уникальных значений в столбце education
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

В данных присутсвуют дубликаты, связанные с различным регистром написания. Это происходит при нестандартизированном заполнении формы о данных клиента.

In [ ]:
#  Применим метод str.lower() для приведения значений к нижнему регистру
data['education'] = data['education'].str.lower()

In [ ]:
# Посчитаем упоминания уникальных значений в столбце education
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [ ]:
# Посчитаем упоминания уникальных значений в столбце family_status
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [ ]:
#  Применим метод str.lower() для приведения значений к нижнему регистру
data['family_status'] = data['family_status'].str.lower()

In [ ]:
# Посчитаем упоминания уникальных значений в столбце family_status
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [ ]:
# Посчитаем упоминания уникальных значений в столбце gender
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце gender как таковых проблем нет.

In [ ]:
# Посчитаем упоминания уникальных значений в столбце income_type
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

В столбце income_type также проблем не выявлено.

In [ ]:
# Посчитаем упоминания уникальных значений в столбце purpose
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Поскольку в столбце purpose все значения в нижнем регистре с ними также ничего делать не будем.

In [ ]:
# Посчитаем сколько вообще явных дубликатов присутствует в таблице
data.duplicated().sum()

71

Появление дубликатов в данных может быть связано с проблемами при выгрузке данных. 

In [ ]:
# Удалим явные дубликаты
data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
# Проверим удаление дубликатов
data.duplicated().sum()

0

**Вывод**

Проверку начали с того, что проверили столбцы содержащие значения типа object. Нашли неявные дубликаты связанные с заполнением данных в разном регистре. Привели все данные к нижнему регистру и тем самым избавились от дубликатов в столбцах со значениями типа object. Далее проверили данные на наличие явных дубликатов и устранили их.

### Лемматизация

Далее проведем лемматизацию в столбце с целью получения кредита. Использовать будем pymystem3.

In [ ]:
# Импортируем pymystem
from pymystem3 import Mystem
m = Mystem()

In [ ]:
# создадим функцию для лемматизации значения 
def lemmatize_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    return ''.join(lemmas)
# протестируем функцию
test_text = 'цель получения кредита'
lemmatize_purpose(test_text)

'цель получение кредит\n'

In [ ]:
# применим функцию к столбцу purpose и создадим столбец с лемматизированными значениями
data['lemmatize_purpose'] = data['purpose'].apply(lemmatize_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n


Найдем частотность слов после лемматизации:

In [ ]:
from collections import Counter
Counter(data['lemmatize_purpose'])

Counter({'покупка жилье\n': 646,
         'приобретение автомобиль\n': 461,
         'дополнительный образование\n': 460,
         'сыграть свадьба\n': 765,
         'операция с жилье\n': 652,
         'образование\n': 447,
         'на проведение свадьба\n': 768,
         'покупка жилье для семья\n': 638,
         'покупка недвижимость\n': 621,
         'покупка коммерческий недвижимость\n': 661,
         'покупка жилой недвижимость\n': 606,
         'строительство собственный недвижимость\n': 635,
         'недвижимость\n': 633,
         'строительство недвижимость\n': 619,
         'на покупка подержать автомобиль\n': 478,
         'на покупка свой автомобиль\n': 505,
         'операция с коммерческий недвижимость\n': 650,
         'строительство жилой недвижимость\n': 624,
         'жилье\n': 646,
         'операция со свой недвижимость\n': 627,
         'автомобиль\n': 972,
         'заниматься образование\n': 408,
         'сделка с подержанный автомобиль\n': 486,
         'получ

**Вывод**

Для лемматизации мы подключили библиотеку pymystem3. Поскольку метод lemmatize применим лишь к тексту мы создали функцию, которая через метод apply сможет пройтись по столбцу purpose и лемматизировать его значения. Значения лемм сохранили в отдельный столбец датафрейма. Также нашли частотность слов после лемматизации.

### Категоризация данных

Посмотрим на данные еще раз и продумаем как датафрейм можно категоризировать.

In [ ]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n


Категории можно создать по цели кредита, по наличию детей, по уровню дохода. Поскольку столбец с целью уже лемматизировали, создадим функцию которая разделит людей на категории кредита.

In [ ]:
data['lemmatize_purpose'].value_counts()

автомобиль\n                                972
свадьба\n                                   791
на проведение свадьба\n                     768
сыграть свадьба\n                           765
операция с недвижимость\n                   675
покупка коммерческий недвижимость\n         661
операция с жилье\n                          652
покупка жилье для сдача\n                   651
операция с коммерческий недвижимость\n      650
жилье\n                                     646
покупка жилье\n                             646
покупка жилье для семья\n                   638
строительство собственный недвижимость\n    635
недвижимость\n                              633
операция со свой недвижимость\n             627
строительство жилой недвижимость\n          624
покупка недвижимость\n                      621
покупка свой жилье\n                        620
строительство недвижимость\n                619
ремонт жилье\n                              607
покупка жилой недвижимость\n            

Все цели содержат лишь пять категорий: автомобиль, жилье, недвижимость, свадьба, образование. Объединим теперь в эти пять категорий весь датафрейм.

In [ ]:
def category_purpose(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'жилье' in purpose:
        return 'жилье'
    if 'недвижимость' in purpose:
        return 'недвижимость'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'образование' in purpose:
        return 'образование'
# протестируем функцию    
test_category = 'автомобиль для бизнеса'
category_purpose(test_category)

'автомобиль'

In [ ]:
data['category_purpose'] = data['lemmatize_purpose'].apply(category_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose,category_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n,свадьба


**Теперь категоризируем наш датасет по наличию детей:**

In [ ]:
def category_children(children):
    if children > 0:
        return 'есть дети в семье'
    else:
        return 'детей нет'
test_children = 2
category_children(test_children)

'есть дети в семье'

In [ ]:
data['category_children'] = data['children'].apply(category_children)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose,category_purpose,category_children
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n,жилье,есть дети в семье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n,автомобиль,есть дети в семье
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n,жилье,детей нет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n,образование,есть дети в семье
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n,свадьба,детей нет


**Чтобы категоризировать уровень дохода** сделаем 2 категории с уровнем выше медианного значения и уровнем ниже медианного значения.

In [ ]:
median_income = data['total_income'].median()
median_income

142594.0

Добавим теперь категории

In [ ]:
def income_level(income):
    if income > median_income:
        return 'выше медианы'
    else:
        return 'ниже медианы'
    
test_income = 150000
income_level(test_income)

'выше медианы'

In [ ]:
# Применим функцию через apply
data['income_level'] = data['total_income'].apply(income_level)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose,category_purpose,category_children,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n,жилье,есть дети в семье,выше медианы
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n,автомобиль,есть дети в семье,ниже медианы
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n,жилье,детей нет,выше медианы
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n,образование,есть дети в семье,выше медианы
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n,свадьба,детей нет,выше медианы


**Вывод**

Категории можно создать по цели кредита, по наличию детей, по уровню дохода.
Проанализировав уникальные значения и их количество в столбце lemmatize_purpoze пришли к выводу, что все цели содержат пять категорий: автомобиль, жилье, недвижимость, свадьба, образование. Создали функцию, которая сможет через метод apply пройтись по всем строкам в столбце с целью кредита и определить их категории. Результат функции сохранили в отдельный столбец category_purpose. Аналогичным образом поступили с категоризацией по наличию детей и с категоризацией по уровню дохода.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Составим сводную таблицу по наличию детей и возврату кредита в срок:

In [ ]:
data_pivot_children = data.pivot_table(index='category_children', values='debt', aggfunc=['sum', 'count'])
data_pivot_children

,sum,count
,debt,debt
category_children,,
детей нет,1064,14138
есть дети в семье,677,7316


Найдем теперь долю тех кто не вернул кредит в срок:

In [ ]:
data_pivot_children['ratio'] = data_pivot_children['sum'] / data_pivot_children['count']
data_pivot_children

,sum,count,ratio
,debt,debt,
category_children,,,
детей нет,1064,14138,0.075258
есть дети в семье,677,7316,0.092537


**Вывод**

Мы проанализировали долю тех кто имеет детей и не имеет и имел задолженности по кредиту. Разница хоть и не существенная, но те кто имеют детей чаще не возвращают кредиты в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_pivot_family = data.pivot_table(index='family_status', values='debt', aggfunc=['sum', 'count']).reset_index()
data_pivot_family

,family_status,sum,count
,,debt,debt
0,в разводе,85,1195
1,вдовец / вдова,63,959
2,гражданский брак,388,4151
3,женат / замужем,931,12339
4,не женат / не замужем,274,2810


In [ ]:
data_pivot_family['ratio'] = data_pivot_family['sum'] / data_pivot_family['count']
data_pivot_family

,family_status,sum,count,ratio
,,debt,debt,
0,в разводе,85,1195,0.071130
1,вдовец / вдова,63,959,0.065693
2,гражданский брак,388,4151,0.093471
3,женат / замужем,931,12339,0.075452
4,не женат / не замужем,274,2810,0.097509


Отношение в столбце ratio показывает какая доля людей не вернула кредит в срок.

**Вывод**

Чтобы ответить на вопрос взаимосвяхи возврата кредита в срок и семейного положения мы создали сводную таблицу по семейном статусе, нашли сумму по столбцу debt(показывает сколько людей не вернули кредит в срок) и количество строк по столбцу debt (показывает сколько людей брали кредит). Далее найдя отношение этих двух величин мы определили долю тех, кто не вернул кредит в срок. Исходя из полученных результатов чаще других категорий не возвращают кредит в срок лица состоящие в гражданском браке или не женатые/не замужние.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим сводную таблицу по уровню дохода:

In [ ]:
data_pivot_income = data.pivot_table(index='income_level', values='debt', aggfunc=['sum', 'count'])
data_pivot_income

,sum,count
,debt,debt
income_level,,
выше медианы,831,10611
ниже медианы,910,10843


Найдем теперь отношение людей имевших задолженности к тем, кто вернул кредит вовремя:

In [ ]:
data_pivot_income['ratio'] = data_pivot_income['sum'] / data_pivot_income['count']
data_pivot_income

,sum,count,ratio
,debt,debt,
income_level,,,
выше медианы,831,10611,0.078315
ниже медианы,910,10843,0.083925


**Вывод**

Необходимо было понять зависит ли возврат кредита в срок от уровня дохода. Для этого мы категоризировали доход на две категории. И далее просчитали долю тех, кто имел задолженности. В обоих категориях значения практически одиннаковое - 8%. Таким образом, возврат кредита в срок не зависит от уровня дохода.

- Как разные цели кредита влияют на его возврат в срок?

Поскольку мы уже категоризировали цели на кредит, составим сводную таблицу для решения:

In [ ]:
data_pivot_purpose = data.pivot_table(index='category_purpose', values='debt', aggfunc=['sum', 'count'])
data_pivot_purpose

,sum,count
,debt,debt
category_purpose,,
автомобиль,403,4306
жилье,308,4460
недвижимость,474,6351
образование,370,4013
свадьба,186,2324


Найдем теперь отношение людей имевших задолженности к тем, кто вернул кредит вовремя:

In [ ]:
data_pivot_purpose['ratio'] = data_pivot_purpose['sum'] / data_pivot_purpose['count']
data_pivot_purpose

,sum,count,ratio
,debt,debt,
category_purpose,,,
автомобиль,403,4306,0.093590
жилье,308,4460,0.069058
недвижимость,474,6351,0.074634
образование,370,4013,0.092200
свадьба,186,2324,0.080034


**Вывод**

Чтобы ответь на вопрос зависит ли возврат кредита в срок от цели кредита мы создали сводную таблицу по ранее выведенным категориям. Далее нашли долю тех, кто не возвращал деньги в срок. Исходя из значений чаще не возвращают в срок те, кто берут кредит на авто или образование.

## Шаг 4. Общий вывод

В этом проекте нам необходимо было поработать с датасетом и ответь на 4 вопроса.
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Мы начали свое исследование с тщательной предобработки данных. Выявили такие проблемы как пропуски, дубликаты, неявные дубликаты, неверные значения. Проблемы были устранены и мы перешли к ответам на вопросы.

**Исследование показало:**
- Те кто имеют детей чаще не возвращают кредиты в срок
- Люди состоящие в гражданском браке или не женатые/не замужние чаще не возвращают кредиты в срок, чем другие категории семейного положения
- Уровень дохода не влияет на возврат кредита в срок
- Кредиты на автомобили и образование чаще других имеют задолженности.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.